In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv("DATA/39ab16a9-7902-48bb-b20d-b96a0bea37ac/output.csv")

# Define anomaly thresholds
anomaly_thresholds = {
    "arm_left": (20, 160),
    "arm_right": (20, 160),
    "backpose_left": (None, 30),  # No lower limit
    "backpose_right": (None, 30)  # No lower limit
}

# List to store anomalies
anomalies = []

# Iterate over each row
for index, row in df.iterrows():
    # timestamp = row["Timestamp"]  # Assuming there's a timestamp column
    for body_part, (lower, upper) in anomaly_thresholds.items():
        angle = row[body_part]
        if (lower is not None and angle < lower) or (upper is not None and angle > upper):
            anomalies.append({
                # "Timestamp": timestamp,
                "Body Part": body_part,
                "Anomalous Angle": angle
            })

# Convert to DataFrame
anomalies_df = pd.DataFrame(anomalies)

# Save to CSV
anomalies_df.to_csv("anomalies.csv", index=False)

print("Anomaly detection completed! Results saved to anomalies.csv.")


Anomaly detection completed! Results saved to anomalies.csv.


In [6]:
import pandas as pd

# Load dataset
df = pd.read_csv("DATA/39ab16a9-7902-48bb-b20d-b96a0bea37ac/output.csv")

# Define anomaly thresholds
anomaly_thresholds = {
    "arm_left": (20, 160),
    "arm_right": (20, 160),
    "backpose_left": (None, 30),  # No lower limit
    "backpose_right": (None, 30)  # No lower limit
}

# Assume 30 rows = 1 second (e.g., 30 FPS)
df["Second"] = df.index // 30

# List to store anomalies
anomalies = []

# Iterate over each body part
for body_part, (lower, upper) in anomaly_thresholds.items():
    df["is_anomalous"] = (df[body_part] < lower) | (df[body_part] > upper) if lower is not None else (df[body_part] > upper)
    
    # Group by second and filter only anomalous data
    anomalous_data = df[df["is_anomalous"]].groupby("Second")[body_part].median().reset_index()
    
    # Store anomalies
    for _, row in anomalous_data.iterrows():
        anomalies.append({
            "Second": row["Second"],
            "Body Part": body_part,
            "Median Anomalous Angle": row[body_part]
        })

# Convert to DataFrame
anomalies_df = pd.DataFrame(anomalies)

# Save to CSV
anomalies_df.to_csv("anomalies.csv", index=False)

print("Anomaly detection completed! Results saved to anomalies.csv.")


Anomaly detection completed! Results saved to anomalies.csv.


In [7]:
import pandas as pd

# Load dataset
df = pd.read_csv("DATA/39ab16a9-7902-48bb-b20d-b96a0bea37ac/output.csv")

# Define anomaly thresholds
anomaly_thresholds = {
    "arm_left": (20, 160),
    "arm_right": (20, 160),
    "backpose_left": (None, 30),  # No lower limit
    "backpose_right": (None, 30)  # No lower limit
}

# Assume 30 rows = 1 second (e.g., 30 FPS)
df["Second"] = df.index // 30

# List to store anomalies
anomalies = []

# Group by second to compute median of each body part
grouped_df = df.groupby("Second").median().reset_index()

def find_anomalous_seconds(grouped_df, body_part, lower, upper):
    anomalous_seconds = []
    prev_anomalous = False
    selected_second = None
    
    for _, row in grouped_df.iterrows():
        is_anomalous = (row[body_part] < lower) | (row[body_part] > upper) if lower is not None else (row[body_part] > upper)
        
        if is_anomalous:
            if not prev_anomalous:
                selected_second = row["Second"]
            prev_anomalous = True
        else:
            if prev_anomalous and selected_second is not None:
                anomalous_seconds.append(selected_second)
            prev_anomalous = False
            selected_second = None
    
    if prev_anomalous and selected_second is not None:
        anomalous_seconds.append(selected_second)
    
    return anomalous_seconds

# Iterate over each body part
for body_part, (lower, upper) in anomaly_thresholds.items():
    anomalous_seconds = find_anomalous_seconds(grouped_df, body_part, lower, upper)
    
    for second in anomalous_seconds:
        anomalies.append({
            "Second": second,
            "Body Part": body_part,
            "Median Anomalous Angle": grouped_df[grouped_df["Second"] == second][body_part].values[0]
        })

# Convert to DataFrame
anomalies_df = pd.DataFrame(anomalies)

# Save to CSV
anomalies_df.to_csv("anomalies.csv", index=False)

print("Anomaly detection completed! Results saved to anomalies.csv.")

Anomaly detection completed! Results saved to anomalies.csv.


In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("DATA/39ab16a9-7902-48bb-b20d-b96a0bea37ac/output.csv")

# Define anomaly thresholds
anomaly_thresholds = {
    "arm_left": (20, 160),
    "arm_right": (20, 160),
    "backpose_left": (None, 30),  # No lower limit
    "backpose_right": (None, 30)  # No lower limit
}

# Assume 30 rows = 1 second (e.g., 30 FPS)
df["Second"] = df.index // 30

# List to store anomalies
anomalies = []

def find_anomalous_seconds(df, body_part, lower, upper):
    anomalous_seconds = []
    prev_anomalous = False
    selected_second = None
    temp_rows = []
    
    for _, row in df.iterrows():
        is_anomalous = (row[body_part] < lower) | (row[body_part] > upper) if lower is not None else (row[body_part] > upper)
        
        if is_anomalous:
            temp_rows.append(row)
            if not prev_anomalous:
                selected_second = row["Second"]
            prev_anomalous = True
        else:
            if prev_anomalous and temp_rows:
                median_second = temp_rows[len(temp_rows) // 2]["Second"]
                anomalous_seconds.append((median_second, temp_rows[len(temp_rows) // 2]))
            prev_anomalous = False
            selected_second = None
            temp_rows = []
    
    if prev_anomalous and temp_rows:
        median_second = temp_rows[len(temp_rows) // 2]["Second"]
        anomalous_seconds.append((median_second, temp_rows[len(temp_rows) // 2]))
    
    return anomalous_seconds

# Iterate over each body part
for body_part, (lower, upper) in anomaly_thresholds.items():
    anomalous_seconds = find_anomalous_seconds(df, body_part, lower, upper)
    
    for second, row in anomalous_seconds:
        anomalies.append({
            "Second": second,
            "Body Part": body_part,
            "Anomalous Angle": row[body_part]
        })

# Convert to DataFrame
anomalies_df = pd.DataFrame(anomalies)

# Save to CSV
anomalies_df.to_csv("anomalies.csv", index=False)

print("Anomaly detection completed! Results saved to anomalies.csv.")

Anomaly detection completed! Results saved to anomalies.csv.


In [10]:
import psycopg2

conn = psycopg2.connect(database="postgres",
                        host="localhost",
                        user="postgres",
                        password="zano0911",
                        port="5432")

In [ ]:
cursor = conn.cursor()

# Create table contain second, body part, and anomalous angle
cursor.execute("""
CREATE TABLE anomalies (
    second INT,
    body_part VARCHAR(50),
    anomalous_angle FLOAT
)
""")


In [12]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="coco8-pose.yaml", epochs=100, imgsz=640)

Ultralytics 8.3.74  Python-3.12.1 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
engine\trainer: task=pose, mode=train, model=yolo11n-pose.pt, data=coco8-pose.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|██████████| 334k/334k [00:00<00:00, 5.60MB/s]
Unzipping C:\1backup zano 2\TMMIN\AI Ergonomic\backend\datasets\coco8-pose.zip to C:\1backup zano 2\TMMIN\AI Ergonomic\backend\datasets\coco8-pose...: 100%|██████████| 27/27 [00:00<00:00, 1421.17file/s]

Dataset download success  (1.5s), saved to C:\1backup zano 2\TMMIN\AI Ergonomic\backend\datasets



100%|██████████| 755k/755k [00:00<00:00, 6.29MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                


...

    100/100         0G     0.5196      1.409     0.2539     0.3872     0.9455          7        640: 100%|██████████| 1/1 [00:01<00:00,  1.89s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.50it/s]

                   all          4         14          1      0.345      0.559      0.292      0.392      0.139      0.108     0.0497



100 epochs completed in 0.084 hours.
Optimizer stripped from runs\pose\train\weights\last.pt, 6.1MB
Optimizer stripped from runs\pose\train\weights\best.pt, 6.1MB

Validating runs\pose\train\weights\best.pt...
Ultralytics 8.3.74  Python-3.12.1 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.76it/s]


                   all          4         14      0.887      0.857      0.918      0.734          1        0.7      0.769      0.377
Speed: 2.3ms preprocess, 125.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\pose\train


In [14]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load an official model
model = YOLO("runs/pose/train/weights/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

Ultralytics 8.3.74  Python-3.12.1 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs


val: Scanning C:\1backup zano 2\TMMIN\AI Ergonomic\backend\datasets\coco8-pose\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]


                   all          4         14      0.887      0.857      0.918      0.734          1        0.7      0.769      0.377
Speed: 3.3ms preprocess, 106.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs\pose\val


array([    0.73351])

In [15]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load an official model
model = YOLO("runs/pose/train/weights/best.pt")  # load a custom model

# Predict with the model
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

100%|██████████| 134k/134k [00:00<00:00, 1.88MB/s]


image 1/1 c:\1backup zano 2\TMMIN\AI Ergonomic\backend\bus.jpg: 640x480 4 persons, 129.0ms
Speed: 6.0ms preprocess, 129.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


In [16]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load an official model
model = YOLO("runs/pose/train/weights/best.pt")   # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered

Ultralytics 8.3.74  Python-3.12.1 torch-2.6.0+cpu CPU (12th Gen Intel Core(TM) i5-12500H)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs


val: Scanning C:\1backup zano 2\TMMIN\AI Ergonomic\backend\datasets\coco8-pose\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.35it/s]


                   all          4         14      0.887      0.857      0.918      0.734          1        0.7      0.769      0.377
Speed: 3.2ms preprocess, 92.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\pose\val2


In [17]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model="runs/pose/train/weights/best.pt", data="coco8.yaml", imgsz=640, half=False, device=0)

Ultralytics 8.3.74  Python-3.12.1 torch-2.6.0+cpu 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
